In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

# Part 1: AlexNet from scratch

In [6]:

# Define the AlexNet Architecture
class AlexNet(nn.Module):
    def __init__(self, num_classes=100):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [3]:
# Data Preparation
def get_dataloaders(batch_size=64):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    train_dataset = datasets.ImageFolder(root='archive/train', transform=transform)
    valid_dataset = datasets.ImageFolder(root='archive/valid', transform=transform)
    test_dataset = datasets.ImageFolder(root='archive/test', transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, valid_loader, test_loader


In [1]:
def train_model(model, train_loader, valid_loader, device, num_epochs=10):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        # Training loop with accuracy calculation
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch'):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        train_accuracy = 100 * correct_train / total_train
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Train Accuracy: {train_accuracy:.2f} %')
        
        # Validation loop with accuracy calculation
        model.eval()
        correct_valid = 0
        total_valid = 0
        running_loss_valid = 0.0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_loss_valid += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total_valid += labels.size(0)
                correct_valid += (predicted == labels).sum().item()

        valid_accuracy = 100 * correct_valid / total_valid
        valid_loss = running_loss_valid / len(valid_loader)
        print(f'Validation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_accuracy:.2f} %')

    print('Finished Training')


In [14]:
def test_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f} %')

In [4]:
train_loader, valid_loader, test_loader = get_dataloaders(batch_size=64)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AlexNet(num_classes=100)

In [7]:
train_model(model, train_loader, valid_loader, device, num_epochs=10)

Epoch 1/10: 100%|██████████| 211/211 [43:46<00:00, 12.45s/batch]


Epoch 1, Loss: 4.565894818418964, Train Accuracy: 1.41 %
Validation Loss: 4.4309, Validation Accuracy: 1.80 %


Epoch 2/10: 100%|██████████| 211/211 [43:53<00:00, 12.48s/batch]


Epoch 2, Loss: 4.253803169557834, Train Accuracy: 3.48 %
Validation Loss: 4.0711, Validation Accuracy: 5.00 %


Epoch 3/10: 100%|██████████| 211/211 [1:29:21<00:00, 25.41s/batch]   


...

Epoch 8, Loss: 3.1173960043920723, Train Accuracy: 21.92 %
Validation Loss: 3.0513, Validation Accuracy: 24.80 %


Epoch 9/10: 100%|██████████| 211/211 [42:20<00:00, 12.04s/batch]


Epoch 9, Loss: 2.9519548054555016, Train Accuracy: 25.32 %
Validation Loss: 3.0534, Validation Accuracy: 25.60 %


Epoch 10/10: 100%|██████████| 211/211 [37:52<00:00, 10.77s/batch]


Epoch 10, Loss: 2.799468681145618, Train Accuracy: 28.38 %
Validation Loss: 2.9846, Validation Accuracy: 24.00 %
Finished Training


In [8]:
test_model(model, test_loader, device)

Test Accuracy: 31.20 %


# Part 2: AlexNet pretrained

### Transfer learning strategy:

1. **Modify the Classifier**:
    Since AlexNet is pre-trained on ImageNet, which has 1000 classes, we'll adjust the final layer to the number of classes you have in our dataset - 100 classes. 
    
2. **Freeze Feature Layers**: 
    Freeze the parameters of the feature extraction layers to avoid updating them during training, which can be done by setting `requires_grad = False`.

3. **Define the Loss Function and Optimizer**: 
    Select a loss function Adam, which will only update the parameters of the unfrozen layers.

4. **Fine-Tuning**: 
    Train the model on your new dataset, optionally using techniques such as learning rate decay, data augmentation, or regularization to improve performance.



In [8]:
# Load the pre-trained AlexNet
alexnet = models.alexnet(pretrained=True)

/Users/Lanado/anaconda3/envs/my_env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/Lanado/anaconda3/envs/my_env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
# Freeze the features layers
for param in alexnet.features.parameters():
    param.requires_grad = False

In [10]:
num_classes = 100
alexnet.classifier[6] = nn.Linear(alexnet.classifier[6].in_features, num_classes)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.classifier.parameters(), lr=0.001)

In [12]:
# Number of epochs
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    alexnet.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = alexnet(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation loop
    alexnet.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = alexnet(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Print statistics
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {running_loss/len(train_loader):.4f}, '
          f'Validation Accuracy: {100 * correct/total:.2f}%')

print('Finished Training')

Epoch [1/10], Train Loss: 1.9995, Validation Accuracy: 70.20%
Epoch [2/10], Train Loss: 1.2342, Validation Accuracy: 77.00%
Epoch [3/10], Train Loss: 1.0467, Validation Accuracy: 74.80%
Epoch [4/10], Train Loss: 0.9254, Validation Accuracy: 75.60%
Epoch [5/10], Train Loss: 0.8890, Validation Accuracy: 77.60%
Epoch [6/10], Train Loss: 0.8696, Validation Accuracy: 77.80%
Epoch [7/10], Train Loss: 0.7730, Validation Accuracy: 80.00%
Epoch [8/10], Train Loss: 0.7725, Validation Accuracy: 79.40%
Epoch [9/10], Train Loss: 0.8670, Validation Accuracy: 77.60%
Epoch [10/10], Train Loss: 0.8063, Validation Accuracy: 75.80%
Finished Training


In [15]:
test_model(alexnet, test_loader, device)

Test Accuracy: 79.40 %


### Discussion 

**AlexNet from scratch:**
- It starts with a high training loss and low accuracy.
- There's a consistent improvement in both training loss and accuracy over epochs.
- The validation accuracy also improves, but it seems to plateau around 24-25% by the end of 10 epochs.

**Pretrained AlexNet:**
- It begins with much lower training loss and significantly higher validation accuracy.
- There is an improvement in the training loss and validation accuracy initially, but it starts to plateau and slightly fluctuate towards the end of the training.
- The validation accuracy is substantially higher, peaking at 80% and staying around the mid to high 70s.


The pretrained AlexNet model has a significant advantage in starting from a point where it has already learned a variety of features from a large and diverse dataset (ImageNet). This allows it to adapt to the new task much more quickly. The model trained from scratch is learning to identify features and classify images simultaneously. This is a much more difficult task and explains the slower improvement in performance. The pretrained model only needs to adjust to the specific features of the new dataset.

The pretrained model achieves much higher accuracy and lower loss at a much quicker pace, highlighting the benefits of transfer learning, especially when the dataset used for the initial training is large and varied.

However, the pretrained model shows signs of overfitting as the validation accuracy starts to decrease while the training loss continues to decrease. This could be due to the model beginning to memorize the training data rather than learning to generalize from it. Regularization techniques, more data, or data augmentation could be used to combat this.

Training from scratch is computationally expensive and time-consuming - batch times are significant. Transfer learning is not just more effective in terms of accuracy, it's also more efficient.

Lastly, for generalization, the features learned by the model on ImageNet contain a lot of general information that is useful for a wide range of visual recognition tasks. This is why the pretrained model generalizes better to your validation set.